In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [2]:
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.model_selection import backtesting_forecaster
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas.tseries import offsets
from skforecast import __version__
print(__version__)


0.10.1


In [3]:
# simulate a time series
n = 365
y = pd.Series(
    data=np.arange(n),
    index = pd.date_range(start='2000-01-01', periods=n, freq='d')
)
y

2000-01-01      0
2000-01-02      1
2000-01-03      2
2000-01-04      3
2000-01-05      4
             ... 
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Length: 365, dtype: int64

In [4]:
forecaster = ForecasterEquivalentDate(offset=10, n_offsets=2)
forecaster.fit(y=y)
forecaster

ForecasterEquivalentDate 
Offset: 10 
Number of offsets: 2 
Aggregation function: mean 
Window size: 20 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-11-02 15:21:07 
Last fit date: 2023-11-02 15:21:07 
Skforecast version: 0.10.1 
Python version: 3.11.4 
Forecaster id: None 

In [5]:
forecaster.predict(steps=15)

2000-12-31    350.0
2001-01-01    351.0
2001-01-02    352.0
2001-01-03    353.0
2001-01-04    354.0
2001-01-05    355.0
2001-01-06    356.0
2001-01-07    357.0
2001-01-08    358.0
2001-01-09    359.0
2001-01-10    350.0
2001-01-11    351.0
2001-01-12    352.0
2001-01-13    353.0
2001-01-14    354.0
Freq: D, Name: pred, dtype: float64

In [6]:
metric, predictions_1 = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    initial_train_size=20,
    steps=15,
    metric='mean_absolute_error',
    verbose=False
)

  0%|          | 0/23 [00:00<?, ?it/s]

In [7]:
forecaster = ForecasterEquivalentDate(
    offset=offsets.Day(10),
    n_offsets=2
)
forecaster.fit(y=y)
forecaster

ForecasterEquivalentDate 
Offset: <10 * Days> 
Number of offsets: 2 
Aggregation function: mean 
Window size: 20 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-11-02 15:21:07 
Last fit date: 2023-11-02 15:21:07 
Skforecast version: 0.10.1 
Python version: 3.11.4 
Forecaster id: None 

In [8]:
forecaster.predict(steps=15)

2000-12-31    350.0
2001-01-01    351.0
2001-01-02    352.0
2001-01-03    353.0
2001-01-04    354.0
2001-01-05    355.0
2001-01-06    356.0
2001-01-07    357.0
2001-01-08    358.0
2001-01-09    359.0
2001-01-10    350.0
2001-01-11    351.0
2001-01-12    352.0
2001-01-13    353.0
2001-01-14    354.0
Freq: D, Name: pred, dtype: float64

In [9]:
forecaster = ForecasterEquivalentDate(
    offset = offsets.Day(10),
    n_offsets=2
)

metric, predictions_2 = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    initial_train_size=20,
    steps=15,
    metric='mean_absolute_error',
    verbose=False,
    n_jobs=1
)
predictions_2

  0%|          | 0/23 [00:00<?, ?it/s]

,pred
2000-01-21,5.0
2000-01-22,6.0
2000-01-23,7.0
2000-01-24,8.0
2000-01-25,9.0
...,...
2000-12-26,335.0
2000-12-27,336.0
2000-12-28,337.0
2000-12-29,338.0


In [10]:
pd.testing.assert_frame_equal(predictions_1, predictions_2)

In [96]:
y = pd.Series(
        data = np.array([0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
                         0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
                         0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
                         0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
                         0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
                         0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
                         0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
                         0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
                         0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
                         0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453]
            ),
        index=pd.date_range(start='2000-01-01', periods=50, freq='d'),
        name = 'y'
    )

last_window = pd.Series(
        data = np.array([0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
                         0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453]),
        index = pd.date_range(start='2000-08-05', periods=10, freq='D'),
        name = 'y'
    )

last_window.tail(10)

2000-08-05    0.737995
2000-08-06    0.182492
2000-08-07    0.175452
2000-08-08    0.531551
2000-08-09    0.531828
2000-08-10    0.250455
2000-08-11    0.483034
2000-08-12    0.985560
2000-08-13    0.519485
2000-08-14    0.612895
Freq: D, Name: y, dtype: float64

In [97]:
forecaster = ForecasterEquivalentDate(
        offset=5,
        n_offsets=2
    )
forecaster.fit(y=y)
predictions = forecaster.predict(steps=7, last_window=last_window)
predictions

2000-08-15    0.494225
2000-08-16    0.332763
2000-08-17    0.580506
2000-08-18    0.525518
2000-08-19    0.572361
2000-08-20    0.494225
2000-08-21    0.332763
Freq: D, Name: pred, dtype: float64

In [93]:
(0.612895 + 0.531828)/2

0.5723615

In [98]:
def test_predict_with_int_offset_offset_7_n_offsets_2_using_last_window():
    """
    Test predict method with int offset, offset=1 and n_offsets=1, using last_window.
    """
    last_window = pd.Series(
        data = np.array([0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
                         0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453]),
        index = pd.date_range(start='2000-08-05', periods=10, freq='D'),
        name = 'y'
    )
    forecaster = ForecasterEquivalentDate(
                    offset    = 5,
                    n_offsets = 2
                 )
    forecaster.fit(y=y)
    predictions = forecaster.predict(steps=7, last_window=last_window)
    expected_predictions = pd.Series(
        data = np.array([0.49422539, 0.332763, 0.58050578, 0.52551824, 0.57236106,
                         0.49422539, 0.332763]),
        index = pd.date_range(start='2000-08-15', periods=7, freq='D'),
        name = 'pred'
    )
    pd.testing.assert_series_equal(predictions, expected_predictions)

test_predict_with_int_offset_offset_7_n_offsets_2_using_last_window()

In [63]:
def test_predict_with_int_offset_offset_7_n_offsets_1():
    """
    Test predict method with int offset, offset=7 and n_offsets=1.
    """
    forecaster = ForecasterEquivalentDate(
        offset=7,
        n_offsets=2,
        agg_func=np.mean
    )
    forecaster.fit(y=y)
    predictions = forecaster.predict(steps=7)
    expected_predictions = pd.Series(
        data = np.array([0.42058876, 0.87984916, 0.5973077 , 0.49243547, 0.80475637,
                         0.31755176, 0.46509001]),
        index = pd.date_range(start='2000-02-20', periods=7, freq='D'),
        name = 'pred'
    )
    pd.testing.assert_series_equal(predictions, expected_predictions)

test_predict_with_int_offset_offset_7_n_offsets_1()

In [ ]:
def test_predict_with_int_offset_offset_7_n_offsets_2():
    """
    Test predict method with int offset, offset=7 and n_offsets=2.
    """
    forecaster = ForecasterEquivalentDate(
        offset=7,
        n_offsets=2
    )
    forecaster.fit(y=y)
    predictions = forecaster.predict(steps=5)
    expected_predictions = pd.Series(
        data = np.array([0.41482621, 0.86630916, 0.25045537, 0.48303426, 0.98555979]),
        index = pd.date_range(start='2000-02-20', periods=5, freq='D'),
        name = 'pred'
    )
    pd.testing.assert_series_equal(predictions, expected_predictions)

test_predict_with_int_offset_offset_7_n_offsets_1()

In [70]:
import pytest

def test_predict_exception_with_unfitted_forecaster():
    """
    Test exception when predict method is called before fitting the forecaster.
    """
    forecaster = ForecasterEquivalentDate(offset=1, n_offsets=1)
    with pytest.raises(
        Exception,
        match="This Forecaster instance is not fitted yet. Call `fit` with appropriate arguments before using predict."
        ):
        forecaster.predict(steps=3)

test_predict_with_unfitted_forecaster()